In [3]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [4]:
df = pd.read_csv('/Users/harveys/Desktop/outbound_feb_26.csv')
df.head()

,customer_no,campaign,plan_close_dt,previous_step_at_closure,days_to_plan_close,pkg_no,Package,MOS,order_dt,campaign_series,campaign_year,campaign_start_dt
0,9702,2025 OC TN 5+,11/12/2024,TKT - 2nd contact complete,56,1046,Thursday Nights Plus,2. Subscribers,00:00.0,Thursday Night,2025,2/10/2024
1,9702,2026 - OC Thursday,30/10/2025,TKT - 2nd contact complete,21,1085,Thursday Nights 8-pack,2. Subscribers,17:53.0,Thursday Night,2026,29/9/2025
2,11401,2025 OC TN 5+,7/1/2025,TKT - 3rd contact complete,83,1046,Thursday Nights Plus,2. Subscribers,43:38.0,Thursday Night,2025,2/10/2024
3,11401,2026 - OC Thursday,14/10/2025,TKT - 1st contact complete,5,1046,Thursday Nights Plus,2. Subscribers,43:38.0,Thursday Night,2026,29/9/2025
4,11757,2026 - OC Sat Matinees,3/11/2025,TKT - 2nd contact complete,31,1042,Saturday Matinees,2. Subscribers,00:00.0,Saturday Matinee,2026,29/9/2025


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_no               322 non-null    int64 
 1   campaign                  322 non-null    object
 2   plan_close_dt             322 non-null    object
 3   previous_step_at_closure  321 non-null    object
 4   days_to_plan_close        322 non-null    int64 
 5   pkg_no                    322 non-null    int64 
 6   Package                   322 non-null    object
 7   MOS                       322 non-null    object
 8   order_dt                  322 non-null    object
 9   campaign_series           319 non-null    object
 10  campaign_year             322 non-null    int64 
 11  campaign_start_dt         322 non-null    object
dtypes: int64(4), object(8)
memory usage: 30.3+ KB


In [5]:
df['plan_close_dt'] = pd.to_datetime(df['plan_close_dt'], format='%d/%m/%Y', errors='coerce')
df['campaign_start_dt'] = pd.to_datetime(df['campaign_start_dt'], format='%d/%m/%Y', errors='coerce')
df['close_date'] = df['plan_close_dt'].dt.date
df['days_to_plan_close'] = pd.to_numeric(df['days_to_plan_close'], errors='coerce')
contact_map = {
        'TKT - To start': 0,
        'TKT - 1st contact complete': 1,
        'TKT - 2nd contact complete': 2,
        'TKT - 3rd contact complete': 3,
        'TKT - 4th contact complete': 4,
        'TKT - 5th contact complete': 5
    }
df['contact_count'] = df['previous_step_at_closure'].map(contact_map).fillna(0)
df.head()

,customer_no,campaign,plan_close_dt,previous_step_at_closure,days_to_plan_close,pkg_no,Package,MOS,order_dt,campaign_series,campaign_year,campaign_start_dt,close_date,contact_count
0,9702,2025 OC TN 5+,2024-12-11,TKT - 2nd contact complete,56,1046,Thursday Nights Plus,2. Subscribers,00:00.0,Thursday Night,2025,2024-10-02,2024-12-11,2.0
1,9702,2026 - OC Thursday,2025-10-30,TKT - 2nd contact complete,21,1085,Thursday Nights 8-pack,2. Subscribers,17:53.0,Thursday Night,2026,2025-09-29,2025-10-30,2.0
2,11401,2025 OC TN 5+,2025-01-07,TKT - 3rd contact complete,83,1046,Thursday Nights Plus,2. Subscribers,43:38.0,Thursday Night,2025,2024-10-02,2025-01-07,3.0
3,11401,2026 - OC Thursday,2025-10-14,TKT - 1st contact complete,5,1046,Thursday Nights Plus,2. Subscribers,43:38.0,Thursday Night,2026,2025-09-29,2025-10-14,1.0
4,11757,2026 - OC Sat Matinees,2025-11-03,TKT - 2nd contact complete,31,1042,Saturday Matinees,2. Subscribers,00:00.0,Saturday Matinee,2026,2025-09-29,2025-11-03,2.0


In [5]:
kpi_df = df.groupby('campaign_year').agg(
                Total_Sales=('customer_no', 'count'),
                Avg_Contacts=('contact_count', 'mean'),
                Avg_Days_to_Close=('days_to_plan_close', 'mean'),
                Median_Days_to_Close=('days_to_plan_close', 'median')
            )

kpi_df.head()

,Total_Sales,Avg_Contacts,Avg_Days_to_Close,Median_Days_to_Close
campaign_year,,,,
2025,117,1.401709,28.034188,21.0
2026,205,0.965854,10.726829,1.0


In [8]:
daily_sales = df.groupby(['campaign_year', 'plan_close_dt']).size().reset_index(name='items_sold')
#daily_sales['plan_close_dt'] = pd.to_datetime(daily_sales['plan_close_dt'])

# Get the campaign start date for each year
campaign_start_dates = df.groupby('campaign_year')['campaign_start_dt'].first().reset_index()
campaign_start_dates.columns = ['campaign_year', 'campaign_start_dt']

# Merge to get start dates in the daily sales dataframe
daily_sales = daily_sales.merge(campaign_start_dates, on='campaign_year')

# Calculate days from campaign start
daily_sales['days_from_start'] = (daily_sales['plan_close_dt'] - daily_sales['campaign_start_dt']).dt.days

# Calculate cumulative sales by campaign year
daily_sales['cumulative_sales'] = daily_sales.groupby('campaign_year')['items_sold'].cumsum()

# Drop rows where days_from_start > 200
daily_sales = daily_sales[daily_sales['days_from_start'] <= 200]

# Select and reorder columns
daily_sales = daily_sales[['campaign_year', 'plan_close_dt', 'days_from_start', 'items_sold', 'cumulative_sales']]

daily_sales.head(10)

,campaign_year,plan_close_dt,days_from_start,items_sold,cumulative_sales
0,2025,2024-10-07,5,1,1
1,2025,2024-10-10,8,1,2
2,2025,2024-10-16,14,1,3
3,2025,2024-10-17,15,3,6
4,2025,2024-10-22,20,1,7
5,2025,2024-10-23,21,4,11
6,2025,2024-10-24,22,2,13
7,2025,2024-10-28,26,1,14
8,2025,2024-10-30,28,1,15
9,2025,2024-10-31,29,2,17


In [9]:
x_axis_col = 'plan_close_dt'
x_label = 'Date'
title_suffix = "(Actual Dates)"
fig_daily = px.line(
                daily_sales.sort_values(x_axis_col),
                x=x_axis_col, 
                y='items_sold', 
                color='campaign_year',
                markers=True,
                title=f"Total Sales Velocity {title_suffix}",
                labels={x_axis_col: x_label, 'items_sold': 'Number of Sales', 'campaign_year': 'Year'})

fig_daily.show()

In [ ]:
#pip install --upgrade nbformat

In [10]:
x_axis_col = 'days_from_start'
x_label = 'Days Since Campaign Launch'
title_suffix = "(Aligned by Launch Day)"


fig_cum = px.line(
                    daily_sales, 
                    x=x_axis_col, 
                    y='cumulative_sales', 
                    color='campaign_year',
                    title=f"Cumulative Sales Progression {title_suffix}",
                    labels={x_axis_col: x_label, 'cumulative_sales': 'Total Sales to Date', 'campaign_year': 'Year'})

fig_cum.show()